In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_user_attributes = _hex_json.loads("{}")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"9436b6b7-a5ca-49fe-9162-e06e26369791\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Project Proposal - Dataset Summary - Syed Hashim Raza- Fall 2024\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

The objective of this data collection is to use the dataset and apply predictive ML modeling to see if we can predict games that are potential Game of the Year (GOTY) Candidates.  
The following cells will show the code applied to retrieve the data. 



In [ ]:
#Initially only SteamSpy API was utilized to collect the top 5000 most played games available on Steam. SteamSpy was easier to in this objective to collect the top 5000 games 
#because it will would take too long to retrieve that information from Steam API due to server request limitations and Steam API does not have a get.request method to filter the
#game information extracted based on specific requests such as games with high player count or games owned by number of user.
#SteamSpy was ideal in this regard. 
'''
import requests
import pandas as pd
import time

# Define the SteamSpy API URL
STEAMSPY_API_URL = "https://steamspy.com/api.php"

# Function to get game data from SteamSpy
def get_steamspy_data(request_type, params={}):
    params['request'] = request_type
    response = requests.get(STEAMSPY_API_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

# Fetch game data with different popularity-based endpoints
def fetch_top_games_by_popularity(limit=5000):
    all_games = []

    # Fetch top 100 games based on 2-week players
    top_100_in2weeks = get_steamspy_data('top100in2weeks')
    if top_100_in2weeks:
        all_games.extend(list(top_100_in2weeks.values()))

    # Fetch top 100 games based on total players
    top_100_forever = get_steamspy_data('top100forever')
    if top_100_forever:
        all_games.extend(list(top_100_forever.values()))

    # Fetch games with the 'all' request to fill the rest up to the limit
    offset = 0
    batch_size = 1000  # API supports fetching 1000 games at a time

    while len(all_games) < limit:
        params = {
            'offset': offset
        }
        data = get_steamspy_data('all', params)
        
        if data:
            all_games.extend(list(data.values()))
            offset += batch_size
            time.sleep(2)  # Pause to avoid hitting rate limits
        else:
            break

    # Sort and limit to the top 5000 games based on total players
    sorted_games = sorted(all_games, key=lambda x: x.get('players_forever', 0), reverse=True)
    return sorted_games[:limit]

# Extract specific fields from the game data
def extract_game_data(games_data):
    extracted_data = []

    for game in games_data:
        game_info = {
            'Game Name': game.get('name'),
            'Developer': game.get('developer'),
            'Publisher': game.get('publisher'),
            'Genre': game.get('genre'),
            'Release Date': game.get('release_date'),
            'Owners': game.get('owners'),
            'Players in Last 2 Weeks': game.get('players_2weeks'),
            'Total Players': game.get('players_forever'),
            'Average Playtime (2 Weeks)': game.get('average_2weeks'),
            'Average Playtime (Overall)': game.get('average_forever'),
            'Median Playtime': game.get('median_forever'),
            'Positive Reviews': game.get('positive'),
            'Negative Reviews': game.get('negative'),
            'Average User Score': game.get('score_rank'),
            'Price': game.get('price'),
            'Discount': game.get('discount'),
            'Multiplayer Option': 'Multiplayer' in game.get('tags', []),
            'Single-player': 'Single-player' in game.get('tags', []),
            'Online Play': 'Online PvP' in game.get('tags', []),
            'Co-op Option': 'Co-op' in game.get('tags', []),
            'Platforms': game.get('platforms')
        }
        extracted_data.append(game_info)
    
    return pd.DataFrame(extracted_data)

# Fetch and process the top 5000 games data by popularity
top_5000_games_data = fetch_top_games_by_popularity(limit=5000)
top_5000_games_df = extract_game_data(top_5000_games_data)

# Display the first few rows of the dataframe
print(top_5000_games_df.head())

# Save the dataframe to a CSV file
top_5000_games_df.to_csv("top_5000_steam_games.csv", index=False)
'''

In [ ]:
#In the dataset extracted from the SteamSPY API, the games were missing critical games information likes its tags (genres), release dates and game features like online or 
#multiplayer options. 
#This step was taken to integrate the unique steam app_id of the games so I can use the SteamAPI and extract the missing information and append it to my dataset
# Modify the extraction function to handle list format
'''
def extract_game_data(games_data):
    extracted_data = []

    for game in games_data:
        game_id = game.get('appid')  # Use 'appid' key to get AppID if present
        game_info = {
            'AppID': game_id,
            'Game Name': game.get('name'),
            'Developer': game.get('developer'),
            'Publisher': game.get('publisher'),
            'Genre': game.get('genre'),
            'Release Date': game.get('release_date'),
            'Owners': game.get('owners'),
            'Players in Last 2 Weeks': game.get('players_2weeks'),
            'Total Players': game.get('players_forever'),
            'Average Playtime (2 Weeks)': game.get('average_2weeks'),
            'Average Playtime (Overall)': game.get('average_forever'),
            'Median Playtime': game.get('median_forever'),
            'Positive Reviews': game.get('positive'),
            'Negative Reviews': game.get('negative'),
            'Average User Score': game.get('score_rank'),
            'Price': game.get('price'),
            'Discount': game.get('discount'),
            'Multiplayer Option': 'Multiplayer' in game.get('tags', []),
            'Single-player': 'Single-player' in game.get('tags', []),
            'Online Play': 'Online PvP' in game.get('tags', []),
            'Co-op Option': 'Co-op' in game.get('tags', []),
            'Platforms': game.get('platforms')
        }
        extracted_data.append(game_info)
    
    return pd.DataFrame(extracted_data)

# Re-fetch and process the top 5000 games data by popularity
top_5000_games_data = fetch_top_games_by_popularity(limit=5000)
top_5000_games_df = extract_game_data(top_5000_games_data)

# Display the first few rows of the dataframe
print(top_5000_games_df.head())
'''

In [ ]:
#Finally, this code was used to integrate all the games data with their respective AppID that was missing data in the csv file. 
'''
import requests
import pandas as pd
import time

# Define the Steam API URL and your API key
STEAM_API_KEY = 'E14D0A967B443265598C204216D17739'
STEAM_API_URL = "https://api.steampowered.com/ISteamUserStats/GetSchemaForGame/v2/"
STEAM_APP_DETAILS_URL = "https://store.steampowered.com/api/appdetails"

# Function to fetch detailed game data using the Steam Web API with the API key
def get_steam_game_details(appid, api_key, retries=3):
    params = {
        'key': api_key,
        'appid': appid
    }
    for attempt in range(retries):
        try:
            response = requests.get(STEAM_APP_DETAILS_URL, params={'appids': appid}, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data.get(str(appid), {}).get('success'):
                    return data[str(appid)]['data']
        except requests.exceptions.Timeout:
            print(f"Timeout occurred for AppID: {appid}, retrying ({attempt + 1}/{retries})...")
            time.sleep(2 ** attempt)  # Exponential backoff
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            break
    return None

# Function to update the DataFrame with genres, release dates, and game features
def update_genres_and_features(df, api_key, delay=0.5):
    for index, row in df.iterrows():
        appid = row['AppID']
        if pd.isna(appid):  # Skip rows with missing AppID
            continue

        # Fetch game details from the Steam API using the provided key
        game_details = get_steam_game_details(appid, api_key)
        
        if game_details:
            # Update genre
            genres = game_details.get('genres', [])
            if genres:
                df.at[index, 'Genre'] = ', '.join([genre['description'] for genre in genres])
            
            # Update release date
            release_date = game_details.get('release_date', {}).get('date')
            if release_date:
                df.at[index, 'Release Date'] = release_date
            
            # Update game features
            categories = [cat['description'] for cat in game_details.get('categories', [])]
            df.at[index, 'Multiplayer Option'] = 'Multiplayer' in categories
            df.at[index, 'Single-player'] = 'Single-player' in categories
            df.at[index, 'Online Play'] = 'Online PvP' in categories
            df.at[index, 'Co-op Option'] = 'Co-op' in categories
            
            # Update tags if available
            tags = game_details.get('tags', [])
            if tags:
                df.at[index, 'Tags'] = ', '.join(tags)
        
        # Add delay to avoid rate limits
        time.sleep(delay)
    
    return df

# Load the existing dataset
file_path = r"C:\Users\hashi\Desktop\Rutgers MBS 2024\Semester 2 Fall 2024\Fundamentals of Analytics\Final Project\SteamSpyDataScrape\integrated_top_5000_steam_games.csv"
updated_top_5000_games_df = pd.read_csv(file_path)

# Update the DataFrame with the missing data using the Steam API key
updated_top_5000_games_df = update_genres_and_features(updated_top_5000_games_df, STEAM_API_KEY)

# Save the updated DataFrame to a CSV file
updated_top_5000_games_df.to_csv("reupdated_top_5000_steam_games.csv", index=False)

# Display confirmation message
print("The dataset has been re-updated and saved to 'reupdated_top_5000_steam_games.csv'.")
'''

All the above code was run on Jupyter Notebooks and simply copied here for proof of work. The dataset was also extracted and converted to CSV using Juypter on my machine. The raw dataset was extracted and uploaded here on HEX with the name 'reupdated_top_5000_steam_games.csv'.



In [ ]:
import pandas as pd

df = pd.read_csv("reupdated_top_5000_steam_games.csv")

df

,AppID,Game Name,Developer,Publisher,Genre,Release Date,Owners,Players in Last 2 Weeks,Total Players,Average Playtime (2 Weeks),...,Positive Reviews,Negative Reviews,Average User Score,Price,Discount,Multiplayer Option,Single-player,Online Play,Co-op Option,Platforms
0,570,Dota 2,Valve,Valve,"Action, Strategy, Free To Play","Jul 9, 2013","200,000,000 .. 500,000,000",NaN,NaN,1205,...,1911119,430785,NaN,0,0,False,False,False,True,NaN
1,730,Counter-Strike: Global Offensive,Valve,Valve,"Action, Free To Play","Aug 21, 2012","100,000,000 .. 200,000,000",NaN,NaN,767,...,7274661,1090076,NaN,0,0,False,False,False,False,NaN
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.","Action, Adventure, Massively Multiplayer, Free...","Dec 21, 2017","50,000,000 .. 100,000,000",NaN,NaN,709,...,1440968,1008429,NaN,0,0,False,False,True,False,NaN
3,440,Team Fortress 2,Valve,Valve,"Action, Free To Play","Oct 10, 2007","50,000,000 .. 100,000,000",NaN,NaN,1259,...,991625,130514,NaN,0,0,False,False,False,False,NaN
4,1172470,Apex Legends,Respawn,Electronic Arts,"Action, Adventure, Free To Play","Nov 4, 2020","50,000,000 .. 100,000,000",NaN,NaN,402,...,650439,304487,NaN,0,0,False,False,True,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1211020,Wobbly Life,RubberBandGames,"RubberBandGames, Curve Games",NaN,NaN,"1,000,000 .. 2,000,000",NaN,NaN,0,...,22413,711,NaN,1799,0,False,False,False,False,NaN
4996,450540,"Hot Dogs, Horseshoes & Hand Grenades",RUST LTD.,RUST LTD.,NaN,NaN,"1,000,000 .. 2,000,000",NaN,NaN,58,...,20209,604,NaN,1999,0,False,False,False,False,NaN
4997,585690,Minimalism,PixelMouse,PixelMouse,NaN,NaN,"1,000,000 .. 2,000,000",NaN,NaN,132,...,1276,465,NaN,99,0,False,False,False,False,NaN
4998,220440,DmC: Devil May Cry,Ninja Theory,Capcom,"Action, Adventure","24 Jan, 2013","1,000,000 .. 2,000,000",NaN,NaN,0,...,26393,1946,NaN,2999,0,False,True,False,False,NaN


Now we have the raw dataset loaded up on Hex. I will clean and process the data before it can be worked on. I will check for duplicates and null values first.  Total Players column is completely null, which will be dropped. Owners column is in a ranged value form, which I will change with the average value for easier readability and processing for the ML models to be applied later. A lot of other rows have Nulls or NaN in observations, which I will take a closer look at to see if imputation or simply deletion is the way to go.  



In [ ]:
df.isnull().sum()

AppID                            0
Game Name                        0
Developer                        0
Publisher                        5
Genre                         1049
Release Date                  1088
Owners                           0
Players in Last 2 Weeks       5000
Total Players                 5000
Average Playtime (2 Weeks)       0
Average Playtime (Overall)       0
Median Playtime                  0
Positive Reviews                 0
Negative Reviews                 0
Average User Score            5000
Price                            0
Discount                         0
Multiplayer Option               0
Single-player                    0
Online Play                      0
Co-op Option                     0
Platforms                     5000
dtype: int64

In [ ]:
# The columns 'Players in Last 2 Weeks', 'Total_players', 'Average User Score' and 'Platform' is completely empty. 
# I will be removing those columns 

# Step 2: Remove columns with 100% missing values
columns_to_drop = ['Players in Last 2 Weeks', 'Total Players', 'Average User Score', 'Platforms']
df.drop(columns=columns_to_drop, inplace=True)

# Display updated DataFrame info to verify changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   AppID                       5000 non-null   int64 
 1   Game Name                   5000 non-null   object
 2   Developer                   5000 non-null   object
 3   Publisher                   4995 non-null   object
 4   Genre                       3951 non-null   object
 5   Release Date                3912 non-null   object
 6   Owners                      5000 non-null   object
 7   Average Playtime (2 Weeks)  5000 non-null   int64 
 8   Average Playtime (Overall)  5000 non-null   int64 
 9   Median Playtime             5000 non-null   int64 
 10  Positive Reviews            5000 non-null   int64 
 11  Negative Reviews            5000 non-null   int64 
 12  Price                       5000 non-null   int64 
 13  Discount                    5000 non-null   int6

In [ ]:
# Get the count of missing values in each column
missing_values_per_column = df.isnull().sum()

# Filter to show only columns with missing values
columns_with_missing = missing_values_per_column[missing_values_per_column > 0]

# Display columns with missing values and their counts
print("Columns with missing values:")
print(columns_with_missing)



Columns with missing values:
Publisher          5
Genre           1049
Release Date    1088
dtype: int64


In [ ]:
#Publisher have missing values, which we will impute with 'Unknown'
# Step 3: Impute missing 'Publisher' values
df['Publisher'].fillna('Unknown', inplace=True)

# Verify the changes
missing_values_after_imputation = df['Publisher'].isnull().sum()
print(f"Missing 'Publisher' values after imputation: {missing_values_after_imputation}")

Missing 'Publisher' values after imputation: 0


In [ ]:
#Imputing missing 'Genre' values with 'Unknown'
df['Genre'].fillna('Unknown', inplace=True)

In [ ]:
#Converting and handling 'release date' values
df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')
df['Release Date'].fillna(pd.to_datetime('1900-01-01'), inplace=True)  # Impute with a placeholder date

# Verify changes
missing_values_after_genre_imputation = df['Genre'].isnull().sum()
missing_values_after_release_date_imputation = df['Release Date'].isnull().sum()

(missing_values_after_genre_imputation, missing_values_after_release_date_imputation)

(0, 0)

In [ ]:
# The owners column gives a range (Upper limit and lower limit approximation) of players who own the particular game. 
# I will be taking the average of the owners for simplicity

# Define a function to calculate the average from a range
def calculate_average(range_str):
    # Split the range by '..' and remove commas
    low, high = range_str.replace(',', '').split('..')
    # Convert to integers and calculate the average
    return (int(low) + int(high)) / 2

# Apply the function to the 'owners' column
df['Owners'] = df['Owners'].apply(calculate_average)

# Display the first few rows to verify the changes
df[['Owners']].head()


,Owners
0,350000000.0
1,150000000.0
2,75000000.0
3,75000000.0
4,75000000.0


In [ ]:
# There is 4 columns which specify different variations of multiplayer options
# I will convert that into a single column with True or False, given if any of those 4 columns 
# hold TRUE (multiplayer capability) for simplicity

# Create a new 'multiplayer option' column
df['Multiplayer Feature'] = df[['Multiplayer Option', 'Single-player', 'Online Play', 'Co-op Option']].any(axis=1)

# Drop the original columns as they are now redundant
df.drop(['Multiplayer Option', 'Single-player', 'Online Play', 'Co-op Option'], axis=1, inplace=True)

# Display the first few rows to verify the changes
df.head()


,AppID,Game Name,Developer,Publisher,Genre,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,Negative Reviews,Price,Discount,Multiplayer Feature
0,570,Dota 2,Valve,Valve,"Action, Strategy, Free To Play",2013-07-09,350000000.0,1205,40501,915,1911119,430785,0,0,True
1,730,Counter-Strike: Global Offensive,Valve,Valve,"Action, Free To Play",2012-08-21,150000000.0,767,31069,5255,7274661,1090076,0,0,False
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.","Action, Adventure, Massively Multiplayer, Free...",2017-12-21,75000000.0,709,25204,6415,1440968,1008429,0,0,True
3,440,Team Fortress 2,Valve,Valve,"Action, Free To Play",2007-10-10,75000000.0,1259,12787,330,991625,130514,0,0,False
4,1172470,Apex Legends,Respawn,Electronic Arts,"Action, Adventure, Free To Play",2020-11-04,75000000.0,402,8573,689,650439,304487,0,0,True


In [ ]:
# A few rows in the genre column have non-english symbols in them. We will be identifying them first
# then removing them. 
import re

# Identify and count rows with non-English characters in the 'genre' column

# Define a function to check for non-English characters
def contains_non_english(text):
    return bool(re.search(r'[^\x00-\x7F]', str(text)))

# Apply the function to find rows with non-English characters in 'genre'
non_english_genres = df[df['Genre'].apply(contains_non_english)]

# Count of rows with non-English characters
non_english_count = non_english_genres.shape[0]
print(f"Number of rows with non-English characters in 'Genre': {non_english_count}")

# Display the first few rows with non-English characters
non_english_genres.head()

Number of rows with non-English characters in 'Genre': 38


,AppID,Game Name,Developer,Publisher,Genre,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,Negative Reviews,Price,Discount,Multiplayer Feature
35,1203220,NARAKA: BLADEPOINT,24 Entertainment,NetEase Games Global,"Akční, Dobrodružné, Masivně multiplayerové",1900-01-01,35000000.0,1171,7148,437,210404,90228,0,0,True
90,221100,DayZ,Bohemia Interactive,Bohemia Interactive,"Экшены, Приключенческие игры, Многопользовател...",1900-01-01,7500000.0,803,13094,2290,308187,96489,4999,0,False
151,594570,Total War: WARHAMMER II,"CREATIVE ASSEMBLY, Feral Interactive (Mac), Fe...","SEGA, Feral Interactive (Mac), Feral Interacti...","Acción, Estrategia",2017-09-28,7500000.0,1312,10761,3146,113950,9372,5999,0,False
239,1868140,DAVE THE DIVER,MINTROCKET,MINTROCKET,"Приключенческие игры, Казуальные игры, Ролевые...",1900-01-01,3500000.0,44,1276,961,109115,3166,1399,30,False
409,364360,Total War: WARHAMMER,"CREATIVE ASSEMBLY, Feral Interactive (Linux), ...","SEGA, Feral Interactive (Linux), Feral Interac...","Acción, Estrategia",2016-05-24,3500000.0,284,3713,1509,40361,11504,5999,0,False


In [ ]:
df = df[~df['Genre'].apply(contains_non_english)]

# Verify that rows have been removed
print(f"Rows remaining after removal: {df.shape[0]}")

Rows remaining after removal: 4962


In [ ]:
all_genres = df['Genre'].str.split(', ').explode().unique()

# Sort the unique genres alphabetically
sorted_genres = sorted(all_genres)

# Display all unique genres
sorted_genres

['Actie',
 'Action',
 'Adventure',
 'Animation & Modeling',
 'Audio Production',
 'Aventure',
 'Avontuur',
 'Casual',
 'Course automobile',
 'Design & Illustration',
 'Early Access',
 'Education',
 'Estrategia',
 'Eventyr',
 'Free To Play',
 'Game Development',
 'Gore',
 'Gratis at spille',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Race',
 'Racing',
 'Rol',
 'Rollespil (RPG)',
 'Simuladores',
 'Simulatie',
 'Simulation',
 'Sports',
 'Strategie',
 'Strategy',
 'Unknown',
 'Utilities',
 'Video Production',
 'Violent']

In [ ]:

df['Genre'] = df['Genre'].fillna('').str.split(', ')

#  Define the mapping for non-English tags to their English equivalents
genre_mapping = {
    'Actie': 'Action',
    'Aventure': 'Adventure',
    'Avontuur': 'Adventure',
    'Course automobile': 'Racing',
    'Estrategia': 'Strategy',
    'Eventyr': 'Adventure',
    'Gratis at spille': 'Free To Play',
    'Rol': 'RPG',
    'Rollespil (RPG)': 'RPG',
    'Simuladores': 'Simulation',
    'Simulatie': 'Simulation',
    'Strategie': 'Strategy'
}

# Replace non-English tags in the 'genre' column using the mapping
def translate_genres(genres):
    if isinstance(genres, list):
        return [genre_mapping.get(tag, tag) for tag in genres]
    else:
        return genres  # If it's not a list, return as is

# Apply the function to the 'genre' column
df['Genre'] = df['Genre'].apply(translate_genres)

# Step 4: Verify the changes
df['Genre'].head()


/tmp/ipykernel_12/1553011076.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genre'] = df['Genre'].fillna('').str.split(', ')
/tmp/ipykernel_12/1553011076.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genre'] = df['Genre'].apply(translate_genres)


0                     [Action, Strategy, Free To Play]
1                               [Action, Free To Play]
2    [Action, Adventure, Massively Multiplayer, Fre...
3                               [Action, Free To Play]
4                    [Action, Adventure, Free To Play]
Name: Genre, dtype: object

In [ ]:
# Find duplicate entries in the 'Game Name' column
duplicate_game_names = df[df['Game Name'].duplicated()]

# Count of duplicate 'Game Name' values
duplicate_count = duplicate_game_names.shape[0]

# Display the count of duplicates and some of the duplicate rows (if any)
print(f"Number of duplicate 'Game Name' entries: {duplicate_count}")
duplicate_game_names.head()

Number of duplicate 'Game Name' entries: 3965


,AppID,Game Name,Developer,Publisher,Genre,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,Negative Reviews,Price,Discount,Multiplayer Feature
761,47790,Medal of Honor,Electronic Arts,Electronic Arts,[Action],2010-10-12,1500000.0,0,241,259,7192,2355,1999,0,True
882,209160,Call of Duty: Ghosts,Infinity Ward,Activision,[Unknown],1900-01-01,1500000.0,0,903,362,19424,10970,5999,0,False
1000,570,Dota 2,Valve,Valve,"[Action, Strategy, Free To Play]",2013-07-09,350000000.0,1205,40501,915,1911119,430785,0,0,True
1001,730,Counter-Strike: Global Offensive,Valve,Valve,"[Action, Free To Play]",2012-08-21,150000000.0,767,31069,5255,7274661,1090076,0,0,False
1002,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.","[Action, Adventure, Massively Multiplayer, Fre...",2017-12-21,75000000.0,709,25204,6415,1440968,1008429,0,0,True


In [ ]:
# Remove duplicate entries in the 'Game Name' column, keeping the first occurrence
df = df.drop_duplicates(subset='Game Name', keep='first')

# Display the count of rows after removing duplicates
print(f"Number of rows after removing duplicate 'Game Name' entries: {df.shape[0]}")


Number of rows after removing duplicate 'Game Name' entries: 997


In [ ]:
# Cleaned steam games dataset which I will later use One-Hot Encoding for categorial variables like genre tags so the dataset will be
# ready to apply ML models 
df_clean = pd.read_csv("clean_steam_project_data_v2.csv")
df_clean.head()

,AppID,Game Name,Developer,Publisher,Genre,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,Negative Reviews,Price,Discount,Multiplayer Feature
0,570,Dota 2,Valve,Valve,"['Action', 'Strategy', 'Free To Play']",7/9/2013,350000000,1205,40501,915,1911119,430785,0,0,True
1,730,Counter-Strike: Global Offensive,Valve,Valve,"['Action', 'Free To Play']",8/21/2012,150000000,767,31069,5255,7274661,1090076,0,0,True
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.","['Action', 'Adventure', 'Massively Multiplayer...",12/21/2017,75000000,709,25204,6415,1440968,1008429,0,0,True
3,440,Team Fortress 2,Valve,Valve,"['Action', 'Free To Play']",10/10/2007,75000000,1259,12787,330,991625,130514,0,0,True
4,1172470,Apex Legends,Respawn,Electronic Arts,"['Action', 'Adventure', 'Free To Play']",11/4/2020,75000000,402,8573,689,650439,304487,0,0,True


In [ ]:
# For the Genre column, Multi-label One-Hot encoding will be required. 
# First, extract all unique genres from the dataset.
# Create a new column for each unique genre.
# For each game, set 1 if it has that genre , 0 otherwise.


from ast import literal_eval
df_clean['Genre'] = df_clean['Genre'].apply(literal_eval)

# Get all unique genres
all_genres = set()
for genres in df_clean['Genre']:
    all_genres.update(genres)

# Create new columns for each genre
for genre in all_genres:
    df_clean[f'Genre_{genre}'] = df_clean['Genre'].apply(lambda x: 1 if genre in x else 0)

# Drop the original Genre column
df_encoded = df_clean.drop('Genre', axis=1)
df_encoded.head()

,AppID,Game Name,Developer,Publisher,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,...,Genre_Racing,Genre_Game Development,Genre_Simulation,Genre_Action,Genre_Design & Illustration,Genre_Early Access,Genre_Adventure,Genre_Strategy,Genre_Indie,Genre_Casual
0,570,Dota 2,Valve,Valve,7/9/2013,350000000,1205,40501,915,1911119,...,0,0,0,1,0,0,0,1,0,0
1,730,Counter-Strike: Global Offensive,Valve,Valve,8/21/2012,150000000,767,31069,5255,7274661,...,0,0,0,1,0,0,0,0,0,0
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",12/21/2017,75000000,709,25204,6415,1440968,...,0,0,0,1,0,0,1,0,0,0
3,440,Team Fortress 2,Valve,Valve,10/10/2007,75000000,1259,12787,330,991625,...,0,0,0,1,0,0,0,0,0,0
4,1172470,Apex Legends,Respawn,Electronic Arts,11/4/2020,75000000,402,8573,689,650439,...,0,0,0,1,0,0,1,0,0,0


In [ ]:
# Here, I am keeping the original Release Date column, but extracting the year as a separate column, and converting the months into Season
# into 4 different columns to perform one-hot encoding on them, making the data more useful for ML application

# Convert the 'Release Date' column to datetime format
df_encoded['Release Date'] = pd.to_datetime(df_encoded['Release Date'], errors='coerce')

# Extract year from the 'Release Date' column
df_encoded['Release Year'] = df_encoded['Release Date'].dt.year

# Extract month from the 'Release Date' column and map to seasons
# Spring: Mar-May, Summer: Jun-Aug, Fall: Sep-Nov, Winter: Dec-Feb
df_encoded['Release Season'] = df_encoded['Release Date'].dt.month % 12 // 3 + 1

# Map the season numbers to names
season_mapping = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
df_encoded['Release Season'] = df_encoded['Release Season'].map(season_mapping)

# One-hot encode the 'Release Season' column
df_encoded = pd.get_dummies(df_encoded, columns=['Release Season'], prefix='Season')

# Display the updated DataFrame
df_encoded.head()


,AppID,Game Name,Developer,Publisher,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,...,Genre_Early Access,Genre_Adventure,Genre_Strategy,Genre_Indie,Genre_Casual,Release Year,Season_Fall,Season_Spring,Season_Summer,Season_Winter
0,570,Dota 2,Valve,Valve,2013-07-09,350000000,1205,40501,915,1911119,...,0,0,1,0,0,2013,0,0,1,0
1,730,Counter-Strike: Global Offensive,Valve,Valve,2012-08-21,150000000,767,31069,5255,7274661,...,0,0,0,0,0,2012,0,0,1,0
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",2017-12-21,75000000,709,25204,6415,1440968,...,0,1,0,0,0,2017,0,0,0,1
3,440,Team Fortress 2,Valve,Valve,2007-10-10,75000000,1259,12787,330,991625,...,0,0,0,0,0,2007,1,0,0,0
4,1172470,Apex Legends,Respawn,Electronic Arts,2020-11-04,75000000,402,8573,689,650439,...,0,1,0,0,0,2020,1,0,0,0


The final step will be to create a target label column, which will classify the games in the past years which were nominations/candidates for the game of the year award. From the source through which I acquired the data, there was no clear target label, and for the purpose of my objective, which is to be able to predict Game of the Year candidate games, this approach seemed the most logical. This nomination/candidacy data is gathered via online sources (The game awards) and will be added as a target label corresponding to the respective games. The labels will be binary, meaning 1 will represent a past candidate and 0 will represent NOT a candidate for the game of the year awards. This will allow for simplicity and be able to use supervised machine learning models. 



In [ ]:
goty_nominated_games = {
    "Dragon Age: Inquisition": 1222690,
    "Bayonetta 2": None,  # Not available on Steam
    "Dark Souls II": 335300,
    "Hearthstone": None,  # Not available on Steam
    "Middle-Earth: Shadow of Mordor": 241930,
    "The Witcher 3": 292030,
    "Bloodborne": None,  # Not available on Steam
    "Fallout 4": 377160,
    "Metal Gear Solid V": 287700,
    "Super Mario Maker": None,  # Not available on Steam
    "Overwatch 2": None,  # Not available on Steam
    "Doom": 379720,
    "Inside": 304430,
    "Titanfall 2": 1237970,
    "Uncharted 4": None,  # Not available on Steam
    "The Legend of Zelda": None,  # Not available on Steam
    "Horizon Zero Dawn": 1151640,
    "Persona 5": None,  # Not available on Steam
    "PUBG": 578080,
    "Super Mario Odyssey": None,  # Not available on Steam
    "God of War": 1593500,
    "Assassin's Creed Odyssey": 812140,
    "Celeste": 504230,
    "Marvel's Spider-Man": 1817070,
    "Monster Hunter World": 582010,
    "Red Dead Redemption": 1174180,
    "Sekiro": 814380,
    "Death Stranding": 1190460,
    "Resident Evil 2": 883710,
    "The Outer Worlds": 578650,
    "Super Smash Bros": None,  # Not available on Steam
    "The Last of Us Part II": None,  # Not available on Steam
    "Doom Eternal": 782330,
    "Final Fantasy VII Remake": 1462040,
    "Ghost of Tsushima": 2215430,  
    "Hades": 1145360,
    "It Takes Two": 1426210,
    "Deathloop": 1252330,
    "Metroid Dread": None,  # Not available on Steam
    "Psychonauts 2": 607080,
    "Ratchet & Clank": None,  # Not available on Steam
    "Resident Evil Village": 1196590,
    "A Plague Tale: Requiem": 1182900,
    "God of War: Ragnarök": 2322010,  
    "Horizon Forbidden West": None,  # Not available on Steam
    "Stray": 1332010,
    "Xenoblade Chronicles 3": None,  # Not available on Steam
    "Baldur's Gate 3": 1086940,
    "Alan Wake 2": None,  # Not available on Steam
    "The Legend of Zelda: Tears of the Kingdom": None,  # Not available on Steam
    "Marvel's Spider-Man 2": None,  # Not available on Steam
    "Resident Evil 4": 2050650,
    "Prince of Persia": 277030,
    "Like a Dragon: Infinite Wealth": None,  # Not available on Steam
    "Tekken 8": 1778820,
    "Helldivers 2": 553850,  
    "Baltaro": None,  # Placeholder, not available on Steam
    "Final Fantasy Rebirth": None,  # Not released on Steam yet
    "Elden Ring DLC": 1245620,  # Assuming main game ID
    "BlackMyth Wukong": 1683470,
    "Astro Bot": None,  # Not available on Steam
    "Warhammer 40,000: Space Marines": 55150,
    "Plucky Squire": None,  # Placeholder, not available on Steam
    "Silent Hill 2": 1051480
}


In [ ]:
# Create a set of AppIDs for fast lookup
goty_appids = set(filter(None, goty_nominated_games.values()))

# Add the 'GOTY_Nomination' column based on exact AppID matching
df_encoded['GOTY_Nomination'] = df_encoded['AppID'].apply(lambda x: 1 if x in goty_appids else 0)

# Display the first few rows to verify
df_encoded.head()


,AppID,Game Name,Developer,Publisher,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,...,Genre_Adventure,Genre_Strategy,Genre_Indie,Genre_Casual,Release Year,Season_Fall,Season_Spring,Season_Summer,Season_Winter,GOTY_Nomination
0,570,Dota 2,Valve,Valve,2013-07-09,350000000,1205,40501,915,1911119,...,0,1,0,0,2013,0,0,1,0,0
1,730,Counter-Strike: Global Offensive,Valve,Valve,2012-08-21,150000000,767,31069,5255,7274661,...,0,0,0,0,2012,0,0,1,0,0
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",2017-12-21,75000000,709,25204,6415,1440968,...,1,0,0,0,2017,0,0,0,1,1
3,440,Team Fortress 2,Valve,Valve,2007-10-10,75000000,1259,12787,330,991625,...,0,0,0,0,2007,1,0,0,0,0
4,1172470,Apex Legends,Respawn,Electronic Arts,2020-11-04,75000000,402,8573,689,650439,...,1,0,0,0,2020,1,0,0,0,0


In [ ]:
df_encoded['GOTY_Nomination'].value_counts()

0    967
1     30
Name: GOTY_Nomination, dtype: int64

In [ ]:
# Some of the release dates were missing from the SteamSpy retrieval, so a placeholder data of 1/1/1900 was set. 
# Going to use Steam API to retrieve the missing dates in our dataset and update the placeholder
# This was the code ran on jupyter to make the updates 

'''
import requests
from datetime import datetime
import time

# Load your CSV file
file_path = "C:\\Users\\hashi\\Desktop\\Rutgers MBS 2024\\Semester 2 Fall 2024\\Fundamentals of Analytics\\Final Project\\SteamSpyDataScrape\\SteamSpy_SteamAPI_Final\\final_steam_games_db.csv"
df_clean = pd.read_csv(file_path)

# Steam API key
STEAM_API_KEY = 'E14D0A967B443265598C204216D17739'

# Function to fetch release date from Steam API with retry logic
def fetch_release_date(appid, retries=3):
    url = f"http://store.steampowered.com/api/appdetails?appids={appid}&key={STEAM_API_KEY}"
    for _ in range(retries):
        try:
            response = requests.get(url)
            data = response.json()
            if data[str(appid)]['success']:
                release_date_str = data[str(appid)]['data']['release_date']['date']
                # Convert to standard format
                release_date = datetime.strptime(release_date_str, "%b %d, %Y").strftime("%Y-%m-%d")
                return release_date
        except (KeyError, TypeError, ValueError, requests.RequestException):
            time.sleep(1)  # Wait briefly before retrying
    return None

# Track games whose release dates couldn't be updated
missing_release_dates = []

# Update missing/imputed release dates
for index, row in df_clean.iterrows():
    if row['Release Date'] == '1900-01-01':  # Adjust based on the imputed date format
        real_release_date = fetch_release_date(row['AppID'])
        if real_release_date:
            df_clean.at[index, 'Release Date'] = real_release_date
        else:
            missing_release_dates.append(row['AppID'])

# Save the updated DataFrame
output_path = "C:\\Users\\hashi\\Desktop\\final_steam_games_db_updated.csv"
df_clean.to_csv(output_path, index=False)

# Log missing release dates
print(f"Games missing updated release dates: {missing_release_dates}")
print(f"File saved successfully at: {output_path}")
'''

The updated csv is called 'final_steam_games_db_update_dates.csv', uploaded in files on Hex. 

I will read into that, and update the one-hot encoding.

NOTE: Some of the games are still missing release dates, as the data was probably not available on Steam API. Probably due to being very unpopular or very old. 



In [ ]:
# On this csv with updated release dates, I will apply one-hot encoding on the release dates again to capture the datapoints
df_encoded_v2 = pd.read_csv("final_steam_games_db_updated_dates.csv")
df_encoded_v2.head()

,AppID,Game Name,Developer,Publisher,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,...,Genre_Adventure,Genre_Strategy,Genre_Indie,Genre_Casual,Release Year,Season_Fall,Season_Spring,Season_Summer,Season_Winter,GOTY_Nomination
0,570,Dota 2,Valve,Valve,2013-07-09,350000000,1205,40501,915,1911119,...,0,1,0,0,2013,0,0,1,0,0
1,730,Counter-Strike: Global Offensive,Valve,Valve,2012-08-21,150000000,767,31069,5255,7274661,...,0,0,0,0,2012,0,0,1,0,0
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",2017-12-21,75000000,709,25204,6415,1440968,...,1,0,0,0,2017,0,0,0,1,1
3,440,Team Fortress 2,Valve,Valve,2007-10-10,75000000,1259,12787,330,991625,...,0,0,0,0,2007,1,0,0,0,0
4,1172470,Apex Legends,Respawn,Electronic Arts,2020-11-04,75000000,402,8573,689,650439,...,1,0,0,0,2020,1,0,0,0,0


In [ ]:
import pandas as pd

# Convert the 'Release Date' column to datetime format
df_encoded_v2['Release Date'] = pd.to_datetime(df_encoded_v2['Release Date'], errors='coerce')

# Update the 'Release Year' column with the actual year from 'Release Date'
df_encoded_v2['Release Year'] = df_encoded_v2['Release Date'].dt.year

# Define seasons based on month: Spring (Mar-May), Summer (Jun-Aug), Fall (Sep-Nov), Winter (Dec-Feb)
season_mapping = {
    1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'
}

# Map each month in 'Release Date' to its respective season
df_encoded_v2['Season'] = df_encoded_v2['Release Date'].dt.month.map(season_mapping)

# Reset season columns to 0 before updating them based on the new season values
df_encoded_v2[['Season_Fall', 'Season_Spring', 'Season_Summer', 'Season_Winter']] = 0

# Update one-hot encoding for seasons
df_encoded_v2.loc[df_encoded_v2['Season'] == 'Fall', 'Season_Fall'] = 1
df_encoded_v2.loc[df_encoded_v2['Season'] == 'Spring', 'Season_Spring'] = 1
df_encoded_v2.loc[df_encoded_v2['Season'] == 'Summer', 'Season_Summer'] = 1
df_encoded_v2.loc[df_encoded_v2['Season'] == 'Winter', 'Season_Winter'] = 1

# Drop the temporary 'Season' column used for mapping
df_encoded_v2 = df_encoded_v2.drop(columns=['Season'])

print("Updated one-hot encoding for seasons and release year based on corrected release dates.")


Updated one-hot encoding for seasons and release year based on corrected release dates.


In [ ]:
# Some of the games release dates were till not found. Probable causes are that either they are old, unpopular, and never updated
# on steam's database. Removing those games without dates for temporal integrity in the interest of our model's accuracy
df_encoded_v2 = df_encoded_v2[df_encoded_v2['Release Date'] != '1900-01-01']


In [ ]:
# Count the rows where 'Release Date' is '1900-01-01'
placeholder_count = (df_encoded_v2['Release Date'] == '1900-01-01').sum()

print(f"Number of rows with placeholder release date (1/1/1900): {placeholder_count}")


Number of rows with placeholder release date (1/1/1900): 0


In [ ]:
#Saving the final, cleaned dataset, ready for ML applications. 
df_encoded_v2.to_csv("final_steam_games_db.csv", index=False)

# **Summary of the dataset**

As mentioned earlier, the dataset is sourced from Steam API and SteamSpy using their respective API calls. The final, cleaned dataset contains 933 games on the steam platform. The data includes various attributes that capture game metadata, performance metrics, user engagement and pricing, making it suitable for a predictive modeling. It contains a mix of numerical, categorical converted to binary, and datetime based features, enabling a comprehensive analysis and prediction of game popularity or user engagement. The process of cleaning the raw data is described in the code cells above which showcases data and feature engineering done to reach the final clean dataset saved as 'final_steam_games_db'.

The Attributes (Features) are as follows: 

1. AppID - Unique identifier for each game - Integer - 1 - 999999
2. Game Name - Name of the game - Categorical
3. Developer - Name of the developer of the game - Categorical - 'Dota2, Apex Legends'
4. Publisher - Name of the publisher of the game - Categorical - 'Valve, Electronic Arts'
5. Release Date - Original release date of the game - Date - '2013-07-09'
6. Owners - Estimated number of owners of the game - Integer - 0 - 350,000,000
7. Average Playtime (2 weeks) - Average playtime of a single user in the last 2 weeks - Integer - 0 - 12,000 minutes
8. Average Playtime (Overall) - Total average playtime of a single user of the game - Integer - 0 - 40,000 minutes
9. Median Playtime - Median playtime of a single user of the game - Integer - 0 - 8,000 minutes
10. Positive Reviews - Number of positive reviews - Integer - 0, 7,000,000
11. Negative Reviews - Number of negative reviews - Integer - 0 - 1,500,000
12. Price - Price of the game in cents - Integer - 0 - 100,000 (0 indicate Free To Play)
13. Discount - Discount percentage on the game - Integer - 0 - 100
14. Multiplayer Feature - Whether the game supports multiplayer mode - Boolean - 1 (Yes) or 0 (No)
15. Release Year - Year of game release, extracted from the Release Date - 2007 to 2024
16. Seasons (Fall, Winter, Spring, Summer) - Extracted from Release Date into One-Hot encoded features - 0 or 1
17. Genre (RPG, Indie, Casual, Strategy) - Extracted from Genre tags into multilabel One-Hot encoded - 0 or 1
18. GOTY_Nomination - Binary Variable based on The Game Award (Target Label) - 0 or 1



21 one-hot encoded genre columns in total. 

4 one-hot encoded season features





**Examples of Data Instance (Next Code Cell Below)**

AppID - 570

Game Name - Dota 2

Developer Name - Valve

Publisher Name - Valve

Release Date - 7/9/2013

Owners - 350000000

Average Playtime  (2 weeks) - 1205 (In hours)

Average Playtime (Overall) - 40501 (In hours)

Median Playtime - 915 (In hours)

Positive Reviews - 1911119

Negative Reviews - 430785

Price - 0 (Free to Play)

Discount - 0

Multiplayer Feature - TRUE

Genre_Strategy - 1

Genre_Action - 1

Genre_Free To Play - 1

(Rest of the genre columns are 0 - too many to write down here) 

Release Year - 2013

Season_Fall - 0

Season_Spring - 0

Season_Summer - 1

Season_Winter- 0

GOTY_Nomination - 0



The dataset is complete and well-suited for ML applications, with a variety of attributes that can support classification, regression, or clustering models. The diverse features enable a meaningful prediction and analysis of game trends, user behavior, and other performance metrics. Specifically for my purpose of being able to predict GOTY candidate games, this is a well structured dataset. With 42 features encompassing game attributes, player engagement metrics, and release timings, this dataset enables meaningful analysis of what drives game popularity and critical acclaim. Additionally, the target label derived from 'The Game Awards' and 'PC Gamer' publications enables the dataset to be applicable to supervised machine learning techniques like logistic regression, decision trees, random forest etc. The addition of the target label allows for evaluating the model's performance using metrics like accuracy, precision, recall or F1-score. Without the target label, assessing how well the model is performing in identifying GOTY-worthy games becomes challenging. Having a clear target label helps identifying which features are most strongly correlated with GOTY nominations. 

By leveraging features like owners, average playtime, reviews, genres, release season, and year, the dataset is capable of training models to identify patterns that correlate with high recognition and potential GOTY status. The diversity of features allow for a holistic view of game success factors, making it well-suited for this predictive task. Since the objective is to find out whether this dataset can predict potential Game of the Year (GOTY) Candidates via ML Models, there is 6 games in the dataset which were in the past, winners of the GOTY award, and 30 games in total which were nominated for the Game of the Year (GOTY), which will help in creating a robust prediction model. Currently, the 2024 awards are still pending an announcement with many contender games, some which are in the in the dataset. 





In [ ]:
final_steam_games_db = pd.read_csv("final_steam_games_db.csv")
final_steam_games_db.head()

,AppID,Game Name,Developer,Publisher,Release Date,Owners,Average Playtime (2 Weeks),Average Playtime (Overall),Median Playtime,Positive Reviews,...,Genre_Adventure,Genre_Strategy,Genre_Indie,Genre_Casual,Release Year,Season_Fall,Season_Spring,Season_Summer,Season_Winter,GOTY_Nomination
0,570,Dota 2,Valve,Valve,2013-07-09,350000000,1205,40501,915,1911119,...,0,1,0,0,2013,0,0,1,0,0
1,730,Counter-Strike: Global Offensive,Valve,Valve,2012-08-21,150000000,767,31069,5255,7274661,...,0,0,0,0,2012,0,0,1,0,0
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",2017-12-21,75000000,709,25204,6415,1440968,...,1,0,0,0,2017,0,0,0,1,1
3,440,Team Fortress 2,Valve,Valve,2007-10-10,75000000,1259,12787,330,991625,...,0,0,0,0,2007,1,0,0,0,0
4,1172470,Apex Legends,Respawn,Electronic Arts,2020-11-04,75000000,402,8573,689,650439,...,1,0,0,0,2020,1,0,0,0,0
